In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
from scipy.sparse import hstack

## 1. Load Data  
- Memuat file CSV untuk data pelatihan (`train`), data uji (`test`), dan contoh format pengiriman hasil (`sample submission`).  
- Penting untuk memverifikasi dimensi (jumlah baris & kolom) segera setelah pembacaan, agar tahu data sudah ter-load dengan benar dan lengkap.

In [2]:
# 1. Load data
train_df = pd.read_csv('./train/train.csv')
test_df = pd.read_csv('./test/test.csv')
sample_submission = pd.read_csv('./sample_submission/sample_submission.csv')
print(f"  • train_df shape: {train_df.shape}")
print(f"  • test_df  shape: {test_df.shape}\n")

  • train_df shape: (19579, 3)
  • test_df  shape: (8392, 2)



## 2. Encode Labels

### Tujuan:
Mengubah label kategori (nama penulis seperti "EAP", "HPL", "MWS") menjadi angka, agar bisa diproses oleh algoritma machine learning.

### Penjelasan:
- `LabelEncoder()` dari sklearn digunakan untuk mengonversi label string menjadi angka.
- Contohnya: `EAP -> 0`, `HPL -> 1`, `MWS -> 2`.
- Kita menyimpan `mapping` dari kelas string ke angka agar bisa digunakan lagi nanti saat membuat submission.

### Kenapa ini penting?
Sebagian besar algoritma supervised learning di scikit-learn hanya menerima label dalam bentuk numerik. Encoding label memungkinkan kita menjalankan training dan evaluasi dengan lancar.

In [3]:
# 2. Encode labels
label_encoder = LabelEncoder()
train_df['author_encoded'] = label_encoder.fit_transform(train_df['author'])
mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print(f"  • classes: {label_encoder.classes_}")
print(f"  • mapping: {mapping}\n")

  • classes: ['EAP' 'HPL' 'MWS']
  • mapping: {'EAP': np.int64(0), 'HPL': np.int64(1), 'MWS': np.int64(2)}



## 3. TF–IDF Vectorization (Word + Character)

### Tujuan:
Mengubah teks menjadi representasi numerik agar dapat diproses oleh model machine learning.

### Penjelasan:
- **TF-IDF (Term Frequency – Inverse Document Frequency)** digunakan untuk memberikan bobot pada kata atau karakter berdasarkan frekuensi kemunculannya di dokumen dan seluruh korpus.
- Dua jenis vectorizer digunakan:
  - `TfidfVectorizer` berbasis **kata** (`word`, ngram 1–2) untuk menangkap konteks kata dan frasa umum.
  - `TfidfVectorizer` berbasis **karakter** (`char`, ngram 2–5) untuk menangkap pola mikro seperti gaya penulisan, typo, atau ciri khas pengarang.
- Hasil dari kedua vectorizer digabung menggunakan `hstack()` dari scipy menjadi satu matriks fitur besar.
- Proses yang sama juga diterapkan ke data uji (`test_df`).

### Kenapa ini penting?
Menggabungkan word- dan char-level TF-IDF memperkaya informasi yang didapat dari teks dan meningkatkan performa klasifikasi, terutama ketika pengarang memiliki gaya penulisan yang khas.

In [4]:
# 3. TF-IDF vectorization (Word + Char)
word_vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 2),
    max_features=15000,
    stop_words='english'
)
char_vectorizer = TfidfVectorizer(
    analyzer='char',
    ngram_range=(2, 5),
    max_features=5000
)

X_word = word_vectorizer.fit_transform(train_df['text'])
X_char = char_vectorizer.fit_transform(train_df['text'])
X = hstack([X_word, X_char])
y = train_df['author_encoded']

X_test_word = word_vectorizer.transform(test_df['text'])
X_test_char = char_vectorizer.transform(test_df['text'])
X_test = hstack([X_test_word, X_test_char])

print(f"  • X_word shape: {X_word.shape}")
print(f"  • X_char shape: {X_char.shape}")
print(f"  • Combined X shape: {X.shape}\n")
print(f"  • X_test shape: {X_test.shape}\n")

  • X_word shape: (19579, 15000)
  • X_char shape: (19579, 5000)
  • Combined X shape: (19579, 20000)

  • X_test shape: (8392, 20000)



## 4. Stratified K-Fold Training & Ensembling

### Tujuan:
Melatih model dan mengevaluasinya secara adil dengan pembagian data menggunakan teknik cross-validation, lalu menggabungkan hasil prediksi.

### Penjelasan:
- **StratifiedKFold** membagi data latih menjadi 5 bagian (fold), dengan memastikan distribusi label tetap seimbang di setiap fold (stratifikasi).
- Untuk setiap fold:
  1. Data dilatih pada 4 bagian dan divalidasi pada 1 bagian sisanya.
  2. Dua model digunakan:
     - **Logistic Regression (multinomial)**: Cocok untuk klasifikasi multiclass dan memiliki performa bagus pada data TF-IDF.
     - **Multinomial Naive Bayes**: Model probabilistik yang sederhana tapi sangat cocok untuk data teks.
  3. Hasil prediksi dari kedua model digabung menggunakan **soft voting**, yaitu rata-rata dari probabilitas prediksi.
  4. Evaluasi dilakukan menggunakan **log loss**, yang mengukur seberapa baik model memperkirakan probabilitas yang benar.
- Setelah semua fold selesai, hasil prediksi pada test set dirata-rata untuk semua fold.

### Kenapa ini penting?
- **Cross-validation** memberi estimasi performa yang lebih stabil dan adil daripada satu kali split.
- **Soft voting ensemble** menggabungkan kekuatan dua model yang berbeda sehingga lebih akurat dan tahan terhadap overfitting.
- **Log loss** adalah metrik yang sangat baik untuk tugas klasifikasi probabilistik karena menghukum prediksi yang terlalu yakin tapi salah.

In [5]:
# 4. Model training with Stratified K-Fold
NUM_FOLDS = 5
skf = StratifiedKFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)

test_preds = np.zeros((X_test.shape[0], 3))
val_losses = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"▶ Starting Fold {fold}/{NUM_FOLDS}...")
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    print(f"    • X_train shape: {X_train.shape}, y_train distribution: {np.bincount(y_train)}")
    print(f"    • X_val   shape: {X_val.shape}, y_val   distribution: {np.bincount(y_val)}")

    # Model 1: Logistic Regression
    logreg = LogisticRegression(
        C=10,
        solver='lbfgs',
        multi_class='multinomial',
        max_iter=2000,
        random_state=42
    )
    logreg.fit(X_train, y_train)
    val_pred_logreg = logreg.predict_proba(X_val)
    test_pred_logreg = logreg.predict_proba(X_test)
    print(f"    • LR sample probs (first val row): {val_pred_logreg[0]}")

    # Model 2: Naive Bayes
    nb = MultinomialNB(alpha=0.3)
    nb.fit(X_train, y_train)
    val_pred_nb = nb.predict_proba(X_val)
    test_pred_nb = nb.predict_proba(X_test)
    print(f"    • NB sample probs (first val row): {val_pred_nb[0]}")
    
    # Soft Voting Ensemble (average probability)
    val_pred = (val_pred_logreg + val_pred_nb) / 2
    test_pred = (test_pred_logreg + test_pred_nb) / 2

    loss = log_loss(y_val, val_pred)
    val_losses.append(loss)
    print(f"    ✔ Fold {fold} Log Loss: {loss:.4f}\n")

    test_preds += test_pred / NUM_FOLDS

# 5. Output final average log loss
print(f"\nAverage Log Loss across folds: {np.mean(val_losses):.4f}")

▶ Starting Fold 0/5...
    • X_train shape: (15663, 20000), y_train distribution: [6320 4508 4835]
    • X_val   shape: (3916, 20000), y_val   distribution: [1580 1127 1209]


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


    • LR sample probs (first val row): [6.16404359e-01 3.83249456e-01 3.46185588e-04]
    • NB sample probs (first val row): [0.92308394 0.07561432 0.00130173]
    ✔ Fold 0 Log Loss: 0.3956

▶ Starting Fold 1/5...
    • X_train shape: (15663, 20000), y_train distribution: [6320 4508 4835]
    • X_val   shape: (3916, 20000), y_val   distribution: [1580 1127 1209]


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


    • LR sample probs (first val row): [1.67734809e-04 1.37063414e-03 9.98461631e-01]
    • NB sample probs (first val row): [0.00113132 0.0099582  0.98891048]
    ✔ Fold 1 Log Loss: 0.3722

▶ Starting Fold 2/5...
    • X_train shape: (15663, 20000), y_train distribution: [6320 4508 4835]
    • X_val   shape: (3916, 20000), y_val   distribution: [1580 1127 1209]


...

    • LR sample probs (first val row): [0.99353156 0.00306576 0.00340267]
    • NB sample probs (first val row): [0.98912988 0.00368676 0.00718336]
    ✔ Fold 2 Log Loss: 0.3942

▶ Starting Fold 3/5...
    • X_train shape: (15663, 20000), y_train distribution: [6320 4508 4835]
    • X_val   shape: (3916, 20000), y_val   distribution: [1580 1127 1209]


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


    • LR sample probs (first val row): [0.01541246 0.00528238 0.97930515]
    • NB sample probs (first val row): [0.03661721 0.00228845 0.96109434]
    ✔ Fold 3 Log Loss: 0.3797

▶ Starting Fold 4/5...
    • X_train shape: (15664, 20000), y_train distribution: [6320 4508 4836]
    • X_val   shape: (3915, 20000), y_val   distribution: [1580 1127 1208]


C:\Users\LENOVO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


    • LR sample probs (first val row): [0.38783265 0.29057145 0.3215959 ]
    • NB sample probs (first val row): [0.75880302 0.18406893 0.05712805]
    ✔ Fold 4 Log Loss: 0.3828


Average Log Loss across folds: 0.3849


## 5. Prepare Submission

### Tujuan:
Menyusun prediksi akhir ke dalam format yang dapat diunggah sebagai hasil kompetisi.

### Penjelasan:
- Menggunakan prediksi rata-rata dari seluruh fold.
- Probabilitas setiap kelas (`EAP`, `HPL`, `MWS`) disusun dalam DataFrame sesuai urutan yang diminta.
- File CSV disimpan dengan nama `submission.csv` dan tidak menyertakan index.

### Kenapa ini penting?
Hasil akhir harus sesuai dengan format yang disyaratkan platform kompetisi (seperti Kaggle). Kesalahan format dapat menyebabkan file ditolak atau hasil tidak valid, walaupun prediksi kita benar.

In [6]:
# 6. Prepare submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'EAP': test_preds[:, label_encoder.transform(['EAP'])[0]],
    'HPL': test_preds[:, label_encoder.transform(['HPL'])[0]],
    'MWS': test_preds[:, label_encoder.transform(['MWS'])[0]],
})

submission.to_csv('submission.csv', index=False)
print("✅ Submission saved as submission.csv")

✅ Submission saved as submission.csv
